In [1]:
import numpy as np
import cvxopt
from cvxopt import matrix, solvers

In [2]:
source_train_filename = 'source_train.csv'
target_train_filename = 'target_train.csv'

source_t_data = np.loadtxt(source_train_filename, delimiter=",")
target_t_data = np.loadtxt(source_train_filename, delimiter=",")

#seperating the features and labels in source file.
X_s = source_t_data[:,:-1]
y_s = source_t_data[:, -1]

#seperating the features and labels in target file.
X_t = target_t_data[:,:-1]
y_t = target_t_data[:,-1]

In [3]:
def linear_kernel(x_i, x_j):
    return np.matmul(x_i,x_j.T)

# Normal SVM

In [4]:
#Normal SVM function
def svm(X_s, y_s, c):
  n = X_s.shape[0]
  
  # matrix P in quadratic program is a n x n matrix
  P = np.zeros([n,n])
  # Populate P by quadratic part of dual form function
  for i in range(n):
    for j in range(n):
      P[i, j] = y_s[i] * y_s[j] * (np.matmul(X_s[i].T, X_s[j]))
  P = matrix(P)
  # q is a n-dimensional vector and represents the linear part of dual form equition
  q = np.ones(n)
  # and forming vector of -1's
  q = matrix(q * -1.0)
  # G in the dual form is the linear term 
  g1 = - np.diag(np.ones(n))
  g2 = np.diag(np.ones(n))
  G = matrix(np.vstack((g1, g2)))
  # h part is envolved with C
  h1 = np.zeros(n)
  h2 = np.ones(n) * C
  h = matrix(np.hstack((h1, h2)))
  A = matrix(y_s, (1,n))
  b= matrix(0.0)
  # using solver to solve for alpha
  solver = solvers.qp(P, q, G, h,A, b)
  alpha = solver['x'].T
  
  # w = sum(alpha * y * x)
  w = np.array([[0 , 0]])
  for i in range(len(alpha)):
    w =  w + (alpha[i] * y_s[i] * X_s[i])
    
  return w
 

In [5]:
C = 2
Ws = svm(X_s, y_s, C)
print("\nWeight vector Ws:", Ws)

     pcost       dcost       gap    pres   dres
 0: -8.1205e+01 -1.0391e+03  4e+03  2e+00  7e-15
 1: -5.7800e+01 -4.6479e+02  6e+02  2e-01  5e-15
 2: -4.2008e+01 -1.0665e+02  9e+01  2e-02  9e-15
 3: -4.6545e+01 -6.2542e+01  2e+01  4e-03  5e-15
 4: -4.7692e+01 -5.7163e+01  1e+01  2e-03  4e-15
 5: -4.9530e+01 -5.3925e+01  5e+00  3e-04  4e-15
 6: -5.0734e+01 -5.2314e+01  2e+00  4e-05  4e-15
 7: -5.1252e+01 -5.1478e+01  2e-01  2e-06  5e-15
 8: -5.1338e+01 -5.1381e+01  4e-02  3e-07  5e-15
 9: -5.1357e+01 -5.1359e+01  2e-03  6e-11  7e-15
10: -5.1358e+01 -5.1358e+01  3e-05  8e-13  6e-15
Optimal solution found.

Weight vector Ws: [[2.1770603  2.00811781]]


# Domain Adaptive SVM

In [6]:
def DomainAdaptivesvm(X_t, y_t, C, B, w_s):
  n = X_t.shape[0]
  # matrix P in quadratic program is a n x n matrix
  P = np.zeros([n,n])
  # Populate P by quadratic part of dual form function
  for i in range(n):
    for j in range(n):
      P[i, j] = y_s[i] * y_s[j] * (np.matmul(X_s[i].T, X_s[j]))
  P = matrix(P)
  
  q = np.zeros_like((y_t)) 
  for i in range(n):
    q[i] =  (B * y_t[i] * np.dot(w_s, X_t[i])) - 1.0
  q = matrix (q)

  # alpha <= C and -alpha <= 0 
  temp1 = np.diag(-(np.ones(n)))
  temp2 = np.identity(n)
  G = matrix(np.vstack((temp1, temp2)))
  temp1 = np.zeros(n)
  temp2 = np.ones(n) * C
  h = matrix(np.hstack((temp1, temp2)))
  A = matrix(y_t.reshape(1,-1),(1, n),'d')
  b = matrix(0.0)
  #solving for alpha
  a = solvers.qp(P, q, G, h, A, b)
  a = np.array(a['x'])
  #finding the weights
  # w = sum(alpha * y * x) + B * w_s
  w_t = np.array([[0 , 0]])
    
  for i in range(len(a)):
        
    w_t = w_t + (a[i] * y_t[i] * X_t[i])
        
  w_t = B * w_s + w_t
    
  return w_t

In [7]:
B = 1
W_t = DomainAdaptivesvm(X_t, y_t, C , B, Ws)
print("\nThe value of W_t = ", W_t)

     pcost       dcost       gap    pres   dres
 0: -8.3709e+01 -1.0436e+03  4e+03  2e+00  2e-15
 1: -5.9842e+01 -4.6666e+02  7e+02  2e-01  2e-15
 2: -4.1486e+01 -1.0761e+02  1e+02  3e-02  3e-15
 3: -4.3018e+01 -5.6901e+01  2e+01  4e-03  2e-15
 4: -4.4745e+01 -5.0675e+01  7e+00  2e-03  1e-15
 5: -4.5780e+01 -4.7899e+01  2e+00  2e-04  1e-15
 6: -4.6303e+01 -4.7037e+01  8e-01  6e-05  1e-15
 7: -4.6565e+01 -4.6661e+01  1e-01  2e-16  1e-15
 8: -4.6614e+01 -4.6615e+01  1e-03  2e-16  1e-15
 9: -4.6614e+01 -4.6614e+01  1e-05  2e-16  1e-15
Optimal solution found.

The value of W_t =  [[2.49354344 2.30781347]]


In [8]:
B = 0
W_t = DomainAdaptivesvm(X_t, y_t, C , B, Ws)
print("\nThe value of W_t = ", W_t)

     pcost       dcost       gap    pres   dres
 0: -8.1205e+01 -1.0391e+03  4e+03  2e+00  7e-15
 1: -5.7800e+01 -4.6479e+02  6e+02  2e-01  5e-15
 2: -4.2008e+01 -1.0665e+02  9e+01  2e-02  9e-15
 3: -4.6545e+01 -6.2542e+01  2e+01  4e-03  5e-15
 4: -4.7692e+01 -5.7163e+01  1e+01  2e-03  4e-15
 5: -4.9530e+01 -5.3925e+01  5e+00  3e-04  4e-15
 6: -5.0734e+01 -5.2314e+01  2e+00  4e-05  4e-15
 7: -5.1252e+01 -5.1478e+01  2e-01  2e-06  5e-15
 8: -5.1338e+01 -5.1381e+01  4e-02  3e-07  5e-15
 9: -5.1357e+01 -5.1359e+01  2e-03  6e-11  7e-15
10: -5.1358e+01 -5.1358e+01  3e-05  8e-13  6e-15
Optimal solution found.

The value of W_t =  [[2.1770603  2.00811781]]


Result of B = 0 in domain adaptive svm is as same as the normal svm

In [ ]:
!jupyter nbconvert --to html /content/HW2.ipynb

[NbConvertApp] Converting notebook /content/HW2.ipynb to html
[NbConvertApp] Writing 561881 bytes to /content/HW2.html
